# Example Membership Inference Attack using ART

In this notebook, we will use ART, which trains shadow models to generate a meta-dataset for membership inference and conduct the attack.

In [10]:
import numpy as np
import tensorflow as tf
import art
tf.compat.v1.disable_eager_execution()
print(tf.__version__)
print(art.__version__)
# Set random seed for reproducibility
np.random.seed(123)

2.15.1
1.17.1


## Load CIFAR-10 Data

We'll load the CIFAR-10 data and split it for training the target and shadow models.

In [11]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Split the data for target and shadow models
# Assuming 25% for target model and 75% for shadow models
x_target = x_train[:12500]
y_target = y_train[:12500]
x_shadow = x_train[12500:]
y_shadow = y_train[12500:]

# Further split target data for training and testing
target_train_size = len(x_target) // 2
x_target_train = x_target[:target_train_size]
y_target_train = y_target[:target_train_size]
x_target_test = x_target[target_train_size:]
y_target_test = y_target[target_train_size:]

# Normalize pixel values to be between 0 and 1
x_target_train, x_target_test, x_shadow = x_target_train / 255.0, x_target_test / 255.0, x_shadow / 255.0

## Load Pre-trained Target Model

Load the pre-trained target model using the Keras library.

In [12]:
# Load your pre-trained CNN model
model = tf.keras.models.load_model('../models/simple-cifar10.h5')

## Train Shadow Models

We will train multiple shadow models to simulate the behavior of the target model.

In [15]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout

def create_shadow_model():
    shadow_model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    shadow_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return shadow_model

# Train shadow models
num_shadow_models = 3
shadow_models = []
for _ in range(num_shadow_models):
    shadow_model = create_shadow_model()
    x_shadow_train, x_shadow_test, y_shadow_train, y_shadow_test = train_test_split(x_shadow, y_shadow, test_size=0.5)
    shadow_model.fit(x_shadow_train, y_shadow_train, epochs=5, verbose=True)
    shadow_models.append((shadow_model, (x_shadow_train, y_shadow_train), (x_shadow_test, y_shadow_test)))


Train on 18750 samples
Epoch 1/5
18750/18750 [==============================] - 1s 67us/sample - loss: 1.6703 - accuracy: 0.3919
Epoch 2/5
18750/18750 [==============================] - 1s 79us/sample - loss: 1.3434 - accuracy: 0.5164
Epoch 3/5
18750/18750 [==============================] - 1s 77us/sample - loss: 1.2032 - accuracy: 0.5772
Epoch 4/5
18750/18750 [==============================] - 2s 81us/sample - loss: 1.0926 - accuracy: 0.6201
Epoch 5/5
18750/18750 [==============================] - 2s 84us/sample - loss: 1.0028 - accuracy: 0.6500
Train on 18750 samples
Epoch 1/5
18750/18750 [==============================] - 1s 77us/sample - loss: 1.7073 - accuracy: 0.3803
Epoch 2/5
18750/18750 [==============================] - 1s 76us/sample - loss: 1.3474 - accuracy: 0.5188
Epoch 3/5
18750/18750 [==============================] - 1s 71us/sample - loss: 1.1951 - accuracy: 0.5834
Epoch 4/5
18750/18750 [==============================] - 1s 65us/sample - loss: 1.0766 - accuracy: 0.6229


## Generate Shadow Dataset

In [5]:
# Generate shadow dataset
member_x, member_y, member_predictions = [], [], []
nonmember_x, nonmember_y, nonmember_predictions = [], [], []

for shadow_model, (x_shadow_train, y_shadow_train), (x_shadow_test, y_shadow_test) in shadow_models:
    member_preds = shadow_model.predict(x_shadow_train)
    nonmember_preds = shadow_model.predict(x_shadow_test)
    
    member_x.append(x_shadow_train)
    member_y.append(y_shadow_train)
    member_predictions.append(member_preds)
    
    nonmember_x.append(x_shadow_test)
    nonmember_y.append(y_shadow_test)
    nonmember_predictions.append(nonmember_preds)

member_x = np.concatenate(member_x)
member_y = np.concatenate(member_y)
member_predictions = np.concatenate(member_predictions)

nonmember_x = np.concatenate(nonmember_x)
nonmember_y = np.concatenate(nonmember_y)
nonmember_predictions = np.concatenate(nonmember_predictions)


/home/yanni/.local/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


## Conduct the Attack

We can now use the shadow models to train the attack model for membership inference.

In [8]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
from art.estimators.classification import KerasClassifier

# Wrap the Keras model in an ART classifier
classifier = KerasClassifier(model=model, clip_values=(0, 1))

# Train the black-box membership inference attack
attack = MembershipInferenceBlackBox(classifier, attack_model_type="nn")
attack.fit(member_x, member_y, nonmember_x, nonmember_y, member_predictions, nonmember_predictions)

# Evaluate the attack
member_infer = attack.infer(x_target_train, y_target_train)
nonmember_infer = attack.infer(x_target_test, y_target_test)
# Display results - 1 is 100% success 0 is 0% sc
print("Membership inference on training data (member inference):", member_infer)
print("Membership inference on test data (non-member inference):", nonmember_infer)


/home/yanni/.local/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Membership inference on training data (member inference): [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
Membership inference on test data (non-member inference): [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
